In [5]:
import scipy.interpolate as interp
from ruspy.estimation.estimation_transitions import create_transition_matrix
from ruspy.model_code.cost_functions import calc_obs_costs
from ruspy.model_code.cost_functions import lin_cost
from ruspy.model_code.fix_point_alg import calc_fixp
from config import DATA_DIR
from global_vals_funcs import VAL_RESULTS

In [2]:
# Load data. This step will be replaced when we have different data.
df = performance_dataframe()
df_meaned = df.mean(level=0)

In [ ]:
# Load the probability grid from the data.
grid_probs = pkl.load(open(f"{DATA_DIR}/grid.pkl", "rb"))
samples = pkl.load(open(f"{DATA_DIR}/samples.pkl", "rb"))

In [6]:
samples.shape

(55, 100, 3)

In [17]:
for id_grid in range(0, 57):
    for id_sample in range(100):
        for id_omega in range(len(VAL_STRATS)):
            res = pkl.load(open(f"{DATA_DIR}/validation_results/grid_{id_grid}_sample_{id_sample}_{id_omega}.pkl", "rb"))

In [3]:
sample_cols = ["p_0", "p_1", "p_2"]

In [4]:
np.unique(df.loc[(0, slice(None)), :][sample_cols].values, axis=0, return_counts=True)

(array([[2.22044605e-16, 2.22044605e-16, 1.00000000e+00],
        [2.22044605e-16, 1.81818182e-02, 9.81818182e-01],
        [2.22044605e-16, 3.63636364e-02, 9.63636364e-01],
        [1.81818182e-02, 2.22044605e-16, 9.81818182e-01],
        [1.81818182e-02, 1.81818182e-02, 9.63636364e-01],
        [1.81818182e-02, 3.63636364e-02, 9.45454545e-01],
        [3.63636364e-02, 2.22044605e-16, 9.63636364e-01]]),
 array([65, 15,  3, 12,  2,  1,  2]))

# Evaluation grid

Now creat an evaluation grid on which we will later interpolate the performance and then apply the decision criteria.

In [4]:
grid_probs.shape

(55, 3)

In [6]:
q = grid_probs[0]
p = samples[0, 10]
np.sum(np.multiply(p, np.log(np.divide(p, q))))

0.0153245241694843

In [7]:
np.sum(np.multiply(p, np.log(np.divide(p, q))))

0.0153245241694843

In [8]:
p

array([2.22044605e-16, 1.81818182e-02, 9.81818182e-01])

In [9]:
grid_probs[14]

array([0.1, 0.4, 0.5])

In [10]:
grid_probs[43]

array([0.5, 0.3, 0.2])

In [11]:
# The omega grid only to the maximum selected omega.
omega_grid = np.arange(0, 1, 0.01)
one_dim_prob_grid = np.arange(0, 0.9, 0.01)
one_dim_prob_grid[0] += 0.005
interpol_prob_grid = create_prob_grid(one_dim_prob_grid, 3)
adapted_grid = interpol_prob_grid[(interpol_prob_grid[:, 0] >= 0.05) & (interpol_prob_grid[:, 1] >= 0.05) & (interpol_prob_grid[:, 0] <= 0.9) & (interpol_prob_grid[:, 1] <= 0.9) & (interpol_prob_grid[:, 2] > 0.1)]

# Interpolation and apply criteria

In [12]:
omega_vals = df.loc[(0, slice(None)), list(VAL_STRATS)].to_numpy()

In [13]:
omega_vals

array([[-84149.00772127, -84149.00772127, -84149.00772127, ...,
        -84149.00772127, -84149.00772127, -84149.00772127],
       [-84149.00772127, -84149.00772127, -84149.00772127, ...,
        -84149.00772127, -84149.00772127, -84149.00772127],
       [-84149.00772127, -84149.00772127, -84149.00772127, ...,
        -84149.00772127, -84149.00772127, -84149.00772127],
       ...,
       [-84149.00772127, -84149.00772127, -84149.00772127, ...,
        -84149.00772127, -84149.00772127, -84149.00772127],
       [-84149.00772127, -84149.00772127, -84149.00772127, ...,
        -84149.00772127, -84149.00772127, -84149.00772127],
       [-84149.00772127, -84149.00772127, -84149.00772127, ...,
        -84149.00772127, -84149.00772127, -84149.00772127]])

In [14]:
omega_vals.shape

(100, 11)

In [15]:
np.log(0.005)

-5.298317366548036

In [16]:
omega_interpol_vals = np.zeros((grid_probs.shape[0], len(omega_grid)))
for id_grid, gridpoint in enumerate(grid_probs):
    omega_interpol_vals[id_grid, :] = interp.griddata(VAL_STRATS, omega_vals[id_grid, :], omega_grid, method="linear")

In [17]:
omega_interpol_vals.shape

(55, 100)

In [18]:
final_interpol_vals = np.zeros((adapted_grid.shape[0], len(omega_grid)))
for omega_id in range(len(omega_grid)):
    final_interpol_vals[:, omega_id] = interp.griddata(grid_probs[:, :2], omega_interpol_vals[:, omega_id], adapted_grid[:, :2], method="linear")

## Minimax regret

In [19]:
# omegas_mean = np.mean(omega_interpol_vals, axis=0)
# normalized_mean = filter_normalize(omegas_mean)


In [20]:
# regrets_omega = df_meaned.loc[:, "best"].to_numpy().reshape(55, 1) - omega_interpol_vals
# max_regret = np.max(regrets_omega, axis=0)
# normalized_regret = filter_normalize(max_regret * -1)

In [21]:
get_decision_rule_df()

,months,disc_strategy,exp_value
0,0,0.000000,-46985.962594
1,1000,-5051.336957,-46985.962594
2,2000,-9653.143746,-46985.962594
3,3000,-13812.102103,-46985.962594
4,4000,-17579.902280,-46985.962594
...,...,...,...
96,96000,-53390.028609,-46985.962594
97,97000,-53390.372477,-46985.962594
98,98000,-53390.683862,-46985.962594
99,99000,-53390.965687,-46985.962594
